# Genereren van een metadata bestand voor een nieuw project
Dit script is voor het genereren van een metadata bestand voor een project dat nieuw is aangenomen 
of voor een project waar geen historische data bekend is en die toegevoegd moet worden aan de 
werkwijze voor het automatisch genereren van een storingsanalyse.

De metadata is nodig voor het berekeken van de gemiddelde aantal meldingen/storingen per maand en de TOV-tabellen.

De opbouw van de metadata is (vooralsnog) als volgt:
```
{
    project: projectnaam,
    start_datum: dd-mm-yyy,
    contract_info: {
        tijdsregistratie: True,
        minimale_beschikbaarheid: xx,
        minimale_responsetijd: 04:00:00,
        aanwezige_deelinstallaties: [],
        POO_codes, {}
    }
}
```
In deze opbouw worden enkel de deelinstallaties meegenomen met meldingen > 0. Dit geldt ook voor de storingen.

## Stappen
De onderstaande stappen zijn voor het toevoegen van een metadata bestand zonder enige historische data.

### Stap 1 - Specificeren van het project
Metadata bestanden worden geïdentificeerd op basis van de naam van het project. Wanneer een nieuw 
project wordt toegevoegd, is het dus van belang dat u de correcte/officiële projectnaam gebruikt.

#### Voorbeelden
```python
project = "Coentunnel-tracé"
```
```python
project = "Sluis Eefde"
```

In onderstaande code cell moet u de naam van het project specificeren tussen de leestekens, zoals de 
voobeelden weergeven.

In [1]:
# Pas onderstaande regel aan
project = "Test Project"

### Stap 2 - Start defnieren
In deze stap wordt u gevraagd de maand en het jaartal te specifieceren van de start van het project.
De voor uw rapport correcte gegevens moet u invullen zoals in het onderstaande voorbeeld wordt laten zien. Hier is het van belang dat de maand altijd in twee cijfers wordt uitgedrukt.

```python
maand, jaar = "01", "2021"
```

In [2]:
maand, jaar = "01", "2021"

### Stap 3 - Tijdsregistratie specificeren
Voor sommige projecten is vastgesteld dat zij binnen de storingsanalyse ook een analyse van de tijdsregistratie moeten uitvoeren. Als dit het geval is voor het project dat u wilt toevoegen dan neemt u het eerste voorbeeld over. In het geval dat dit niet van toepassing is, dan neemt u het tweede voorbeeld over.
```python
tijdsregistratie = True
```
```python
tijdsregistratie = False
```

In [4]:
# Pas onderstaande regel aan
tijdsregistratie = False

### Stap 4 - Starten van het script
Wanneer u alle bovenstaande stappen heeft doorlopen, is het tijd om het script te starten. Dit doet u door te navigeren naar de optie ```Kernel``` in de balk boven aan uw scherm. Vervolgens selecteert u de optie ```Restart & Run All```.

# Importeren van de benodigdheden

In [5]:
import os

keep_going = True
while keep_going:
    if os.getcwd().endswith("Automatisering Storingsanalyse"):
        keep_going = False
    else:
        os.chdir('..')  # changes the working dir to a level above the current working dir

import json
import pandas as pd
import numpy as np
from datetime import datetime
from pandas import DataFrame
from typing import Union

path_to_source_folder = 'processes/1. building_metadata_file/source_document'
path_to_resource_folder = 'resources/information_mapping'
path_to_export_folder = "data/metadata"

# Definieren van de functies

In [6]:
"""
Functie voor inlezen van de POO_codes.json doc voor poo_code_overzicht
"""
def read_json(filepath) -> json:
    with open(filepath, 'r') as json_file:
        _data = json.load(json_file)
    return _data

def create_start_date_string(month: Union[str, int], year: Union[str, int]) -> str:
    _month = month if isinstance(month, str) else str(month)
    _year = year if isinstance(year, str) else str(year)
    return month + '_' + year

def check_for_file(path_to_folder: str, file: str) -> bool:
    files_in_folder = os.listdir(path_to_folder)
    if file in files_in_folder:
        raise PermissionError(f"A metadata file with the name '{file}' already exists in the specified folder. Executing this code cell would possibly mean that data is lost. Please create a backup of the file and remove it from the folder '{path_to_folder}' before running this process again.")
    else:
        return True

# Gereed maken van het JSON object

In [7]:
poo_code_overzicht = read_json(filepath=os.path.join(path_to_resource_folder, 'POO_codes.json'))

In [8]:
contract_info = {"tijdsregistratie": str(tijdsregistratie),
                 "minimale_beschikbaarheid": "xx",
                 "minimale_responsetijd": "04:00:00",
                 "aanwezige_deelinstallaties": list(),
                 "POO_codes": poo_code_overzicht}

In [9]:
start_datum = create_start_date_string(month=maand, year=jaar)

In [10]:
json_dict = {"project": project,
             "start_datum": start_datum,
             "contract_info": contract_info}

In [11]:
export_file_name = "metadata_file_{}.json".format(project.lower().replace(' ', '_'))

check_for_file(path_to_folder=path_to_export_folder, file=export_file_name)

with open(os.path.join(path_to_export_folder, export_file_name), 'w') as output_file:
    json.dump(json_dict, output_file)
    
print('Succes!')

Succes!


# Afronding
Nu het script is uitgevoerd zijn er twee uitkomsten;
    
    1. U ziet de terugkoppeling 'Succes!'.
    2. U ziet een PermissionError met daarbij een terugkoppeling.
    
## Afronding na optie 1
Indien u de terugkoppeling ```Succes!``` ziet, is het script succesvol doorlopen en staat er een niet bestand in de map ```Automatisering Storingsanalyse/data/metadata```.

## Afronding na optie 2
Indien u de PermissionError als terugkoppeling ziet, is het van belang dat u dit bericht volgt en vervolgens het script nogmaals probeert uit te voeren.